In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
from datasets import load_dataset

emotions = load_dataset('SetFit/Emotion')

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
type(emotions)

datasets.dataset_dict.DatasetDict

In [12]:
emotions['train']

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 16000
})

In [ ]:
# from nlp import list_datasets

In [ ]:
# !pip install nlp

In [ ]:

# !pip install datasets

In [ ]:
!pip install mlflow 

In [ ]:
train_ds = emotions['train']
train_ds

In [ ]:
emotions.set_format('pandas')
df = emotions['train'][:]
df.head()

In [ ]:
# def label_int2str(row):
#     return emotions['train'].features['label'].int2str(row)

# df['label_name'] = df['label'].apply(label_int2str)
# df.head()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
emotions.reset_format()
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
emotions_encoded['train'].column_names


In [ ]:
import torch
import torch.nn as nn

from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
text = "this is a test"
sample_inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {sample_inputs['input_ids'].size()}\n"
      f"Tokenized text: {tokenizer.convert_ids_to_tokens(sample_inputs['input_ids'][0])}")

In [ ]:
inputs = {k:v.to(device) for k,v in sample_inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)

In [ ]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k, v in batch.items()
    if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state

    return {"hidden state": last_hidden_state[:, 0].cpu().numpy()}

In [ ]:

emotions_encoded.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
emotions_encoded.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])
emotions_hidden = emotions_encoded.map(
    extract_hidden_states, batched=True)

In [ ]:
import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden state"])
X_valid = np.array(emotions_hidden["validation"]["hidden state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
print(X_train.shape, X_valid.shape)




from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

pca = PCA(n_components=2)

x_scaled = MinMaxScaler().fit_transform(X_train)
x_2d = pca.fit_transform(x_scaled)
x_2d.shape

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6
model = (AutoModelForSequenceClassification
        .from_pretrained(model_ckpt, num_labels=num_labels)
        .to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level="error",
    report_to="none"
)

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=emotions_encoded['train'],
#     eval_dataset=emotions_encoded['validation'],
#     tokenizer=tokenizer,
# )

# trainer.train()

In [ ]:
# !pip install mlflow 
import mlflow
from transformers import pipeline

In [ ]:

# mlflow.set_experiment("Distill Bert")

In [ ]:
experiment_name = "Distill Bert"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

In [ ]:
# import mlflow
# from transformers import TrainerCallback

# class MLflowCallback(TrainerCallback):
#     def __init__(self, mlflow_experiment_name, mlflow_run_name):
#         self.mlflow_experiment_name = mlflow_experiment_name
#         self.mlflow_run_name = mlflow_run_name

#     def on_train_begin(self, args, state, control, model=None, **kwargs):
#         mlflow.set_experiment(self.mlflow_experiment_name)
#         mlflow.start_run(run_name=self.mlflow_run_name)
        
#         artifact_uri = "/path/to/artifacts"  # Replace with your actual logic
#         self.artifact_uri = artifact_uri.replace("{artifactUri}", mlflow.active_run().info.artifact_uri)
#     def on_train_end(self, args, state, control, model=None, **kwargs):
#         mlflow.end_run()

#     def on_epoch_end(self, args, state, control, **kwargs):
#         metrics = state.metric
#         for metric_name, metric_value in metrics.items():
#             mlflow.log_metric(metric_name, metric_value)


In [ ]:
mlflow.end_run()

In [ ]:
mlflow.set_tracking_uri("http://http://13.233.157.244:5001/")
# s3://your-bucket-name/mlflow-artifacts

experiment_name = "Distill Bert"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level="error",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=emotions_encoded['train'],
    eval_dataset=emotions_encoded['validation'],
    tokenizer=tokenizer,
)




In [8]:
import warnings
warnings.filterwarnings("ignore")

from mlflow.models import infer_signature

In [ ]:
# trainer.train()
model_checkpoint_path = os.path.join("/kaggle/working", "pytorch_model.bin")
trainer.save_model(model_checkpoint_path)
print('sm')
mlflow.pytorch.autolog()
print('al')
with mlflow.start_run(experiment_id=experiment_id, run_name="Run 11") as run:
    mlflow.log_params({
        "num_train_epochs": 2,
        "learning_rate": 2e-5,
        # ... Other training parameters ...
    })
    for metric_name, metric_value in trainer.evaluate().items():
        mlflow.log_metric(metric_name, metric_value)
#     mlflow.pytorch.autolog()
    print('met')
    mlflow.pytorch.log_model(model, "distilbert_emotions_model",artifact_path="artifacts",code_paths=[models_dir])
    print('lm')
    mlflow.end_run()
print('ok')

In [ ]:
import mlflow
import pytorch_lightning as pl
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn

from transformers import AutoModel


class EmotionModel(pl.LightningModule):
    def __init__(self, model_ckpt):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_ckpt)
        self.tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        self.batch_size = 64
        self.logging_steps = len(emotions_encoded["train"]) // self.batch_size

    def forward(self, batch):
        # Implement the forward pass of the model here
        return self.model(batch["input_ids"], attention_mask=batch["attention_mask"])
    
    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        f1 = f1_score(labels, preds, average="weighted")
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc, "f1": f1}

    def training_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        labels = batch["label"]

        # Calculate accuracy and F1 score only when predictions are available
        if preds is not None and labels is not None:
            acc = accuracy_score(labels.cpu(), preds.cpu())
            f1 = f1_score(labels.cpu(), preds.cpu(), average='weighted')

            self.log('val_loss', loss, on_step=True, on_epoch=True)
            self.log('val_acc', acc, on_step=False, on_epoch=True)
            self.log('val_f1', f1, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return optimizer

    def train_dataloader(self):
        dataset = self._get_dataset("train")  
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )
        return dataloader
    
    def val_dataloader(self):
        dataset = self._get_dataset("validation")  
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
            num_workers=4
        )
        return dataloader
    
    def _get_dataset(self, split):
        emotions = load_dataset('SetFit/Emotion')
        dataset = emotions[split]

        def tokenize_and_prepare(batch):
            tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True)
            return {
                'input_ids': tokenized_batch['input_ids'],
                'attention_mask': tokenized_batch['attention_mask'],
                'label': batch['label']
            }

        tokenized_dataset = dataset.map(tokenize_and_prepare, batched=True, batch_size=None)
        tokenized_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

        return tokenized_dataset

In [ ]:
mlflow.set_tracking_uri("http://13.233.157.244:5001/")
mlflow.pytorch.autolog()
experiment_name = "Distill Bert"
model_ckpt = "distilbert-base-uncased"

emotions = load_dataset('SetFit/Emotion')
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])



In [ ]:
model = EmotionModel(model_ckpt)

trainer = pl.Trainer(
    max_epochs=2,
    devices=2,
    accelerator='gpu'
)

with mlflow.start_run(run_name="Run 8") as run:
    trainer.fit(model)
    pipe = pipeline("text-classification", model=model_name, batch_size=1, tokenizer=model.tokenizer)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from mlflow.data.pandas_dataset import PandasDataset

def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

go_emotions = load_dataset("go_emotions")
data = go_emotions.data

train, valid, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

mapping = {
    0:"admiration",
    1:"amusement",
    2:"anger",
    3:"annoyance",
    4:"approval",
    5:"caring",
    6:"confusion",
    7:"curiosity",
    8:"desire",
    9:"disappointment",
    10:"disapproval",
    11:"disgust",
    12:"embarrassment",
    13:"excitement",
    14:"fear",
    15:"gratitude",
    16:"grief",
    17:"joy",
    18:"love",
    19:"nervousness",
    20:"optimism",
    21:"pride",
    22:"realization",
    23:"relief",
    24:"remorse",
    25:"sadness",
    26:"surprise",
    27:"neutral",
}

n_labels = len(mapping)

def one_hot_encoder(df):
    one_hot_encoding = []
    for i in tqdm(range(len(df))):
        temp = [0]*n_labels
        label_indices = df.iloc[i]["labels"]
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)
    return pd.DataFrame(one_hot_encoding)

train_ohe_labels = one_hot_encoder(train)
valid_ohe_labels = one_hot_encoder(valid)
test_ohe_labels = one_hot_encoder(test)


train = pd.concat([train, train_ohe_labels], axis=1)
valid = pd.concat([valid, valid_ohe_labels], axis=1)
test = pd.concat([test, test_ohe_labels], axis=1)

def inspect_category_wise_data(label, n=5):
    samples = train[train[label] == 1].sample(n)
    sentiment = mapping[label]
    
    print(f"{n} samples from {sentiment} sentiment: \n")
    for text in samples["text"]:
        print(text, end='\n\n')

class GoEmotionDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

class GoEmotionClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoEmotionClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output
tokenizer = transformers.SqueezeBertTokenizer.from_pretrained("squeezebert/squeezebert-uncased", do_lower_case=True)

def build_dataset(tokenizer_max_len):
    train_dataset = GoEmotionDataset(train.text.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoEmotionDataset(valid.text.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    
    return train_dataset, valid_dataset

def build_dataloader(train_dataset, valid_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader

def ret_model(n_train_steps, do_prob):
    model = GoEmotionClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
    return model



bert_model = transformers.SqueezeBertModel.from_pretrained("squeezebert/squeezebert-uncased")

def ret_optimizer(model):

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=5e-5)
    return opt

def ret_scheduler(optimizer, num_train_steps):
    sch = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    return {"auc_micro": auc_micro}
def train_fn(data_loader, model, optimizer, device, scheduler):

    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
    return train_loss
    

def eval_fn(data_loader, model, device):
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = loss_fn(outputs, targets)
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

def log_dataset(dataset_name):
    dataset = load_dataset(dataset_name)  # Load the dataset
    serialized_dataset = dataset.save_to_disk("dataset_directory")  # Save the dataset to a directory
    mlflow.log_artifact("dataset_directory")  # Log the serialized dataset



def trainer():
    log_dataset("go_emotions")
    go_emotions = load_dataset("go_emotions")
    data = go_emotions.data
    
    train, valid, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

    train_dataset, valid_dataset = build_dataset(40)
    train_data_loader, valid_data_loader = build_dataloader(train_dataset, valid_dataset, 64)

    mlflow.log_param("train_dataset_size", len(train))
    mlflow.log_param("valid_dataset_size", len(valid))
    mlflow.log_param("test_dataset_size", len(test))

    mlflow.log_param("dataset_name", "Emotions")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mlflow.log_param("device", device)
    n_train_steps = int(len(train_dataset) / 64 * 10)

    model = ret_model(n_train_steps, 0.3)
    optimizer = ret_optimizer(model)
    scheduler = ret_scheduler(optimizer, n_train_steps)
    model.to(device)
    model = nn.DataParallel(model)
    n_epochs = 2

    best_val_loss = 100
#     with mlflow.start_run(experiment_id=experiment_id, run_name="Run 11") as run:
#         mlflow.pytorch.autolog() 
    for epoch in tqdm(range(n_epochs)):
        train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
        eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)

        auc_score = log_metrics(preds, labels)["auc_micro"]
        avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)
        mlflow.log_metric("AUC", auc_score)
        mlflow.log_metric("Avg Train Loss", avg_train_loss)
        mlflow.log_metric("Avg Valid Loss", avg_val_loss)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "./best_model.pt")  
            print("Model saved as current val_loss is: ", best_val_loss)
        script_dir = os.path.dirname(os.path.abspath('file'))
        models_dir = os.path.join(script_dir)
        conda_dir = os.path.join(script_dir)
        
        model_path = "/kaggle/working/mlruns/best_model.pt"
        mlflow.log_artifact(model_path, artifact_path="best_model")
        mlflow.pytorch.log_model(
            pytorch_model= model,
            registered_model_name="best-base",
            artifact_path="artifacts",
#             input_example=df[['Title', 'Attributes']],
#             conda_env=conda_dir,
            code_paths=[models_dir]
    )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/43410 [00:00<?, ?it/s]

  0%|          | 0/5426 [00:00<?, ?it/s]

  0%|          | 0/5427 [00:00<?, ?it/s]

Some weights of the model checkpoint at squeezebert/squeezebert-uncased were not used when initializing SqueezeBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing SqueezeBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SqueezeBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
import mlflow.pytorch

experiment_name = "Distill Bert"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id
    
    
mlflow.set_tracking_uri("http://13.233.157.244:5001/")
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment_id, run_name="Run 13"):
#     mlflow.pytorch.autolog()
     # Enable automatic logging for PyTorch
#     trained_model=trainer()
    trainer()

In [16]:
mlflow.end_run()

In [51]:
import optuna
import mlflow
import os
import torch
import torch.nn as nn
from tqdm import tqdm

# Your existing functions and imports

# Define the objective function for Optuna hyperparameter tuning
def objective(trial):
    with mlflow.start_run(experiment_id=experiment_id, run_name="Hyperparameter Tuning") as run:
        log_dataset("go_emotions")
        go_emotions = load_dataset("go_emotions")
        data = go_emotions.data
        batch_size=64
        train, valid, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

        train_dataset, valid_dataset = build_dataset(40)
        train_data_loader, valid_data_loader = build_dataloader(train_dataset, valid_dataset, batch_size)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        mlflow.log_param("device", device)

        # Define hyperparameter search space
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
        dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)
        do_prob = trial.suggest_uniform("do_prob", 0.3, 0.5)
        n_epochs = trial.suggest_int("n_epochs", 2, 4)
        # Update hyperparameters
    #     model = ret_model(n_train_steps, dropout_rate)  # Update your model with the new dropout_rate
    #     optimizer = ret_optimizer(model, learning_rate)  # Update your optimizer with the new learning_rate

        n_train_steps = int(len(train_dataset) / 64 * 10)

        model = ret_model(n_train_steps, do_prob)
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        model = nn.DataParallel(model)

        best_val_loss = 100

        # Rest of your training loop
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)

            auc_score = log_metrics(preds, labels)["auc_micro"]
            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)
            mlflow.log_metric("AUC", auc_score)
            mlflow.log_metric("Avg Train Loss", avg_train_loss)
            mlflow.log_metric("Avg Valid Loss", avg_val_loss)
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "./best_model.pt")  
                print("Model saved as current val_loss is: ", best_val_loss)
                model_path = "/kaggle/working/best_model.pt"
                mlflow.log_artifact(model_path, artifact_path="best_model")
            script_dir = os.path.dirname(os.path.abspath('file'))
            models_dir = os.path.join(script_dir, "models")
            conda_dir = os.path.join(script_dir, "dependencies", "conda.yaml")

            # Log the best model with Optuna hyperparameters
            mlflow.pytorch.log_model(
                pytorch_model=model,
                registered_model_name="best-base",
                artifact_path="artifacts",
            )

    # Return the value to minimize (e.g., validation loss)
    return best_val_loss

# Main training code
def trainer():
    # Start Optuna hyperparameter tuning
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30,show_progress_bar=False)  

    best_params = study.best_params
    best_metric = study.best_value

    for param_name, param_value in best_params.items():
        mlflow.log_param(param_name, param_value)

    mlflow.log_metric("best_metric", best_metric)


In [ ]:
import mlflow.pytorch

experiment_name = "Distill Bert"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id
    
    
mlflow.set_tracking_uri("http://13.233.157.244:5001/")
# mlflow.pytorch.autolog()

# with mlflow.start_run(experiment_id=experiment_id, run_name="Run 13"):
# #     mlflow.pytorch.autolog()
#      # Enable automatic logging for PyTorch
# #     trained_model=trainer()
#     
trainer()

[I 2023-08-28 11:47:19,304] A new study created in memory with name: no-name-a44d4a31-d307-41cd-83b8-c42259ea859d


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 679/679 [03:31<00:00,  3.21it/s]

100%|██████████| 85/85 [00:15<00:00,  5.47it/s]


Model saved as current val_loss is:  0.10415135797332314


In [47]:
mlflow.end_run()

In [ ]:
/kaggle/working/best_model.pt